## Simple Notebook for preparing the whole training run in one graph

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
%matplotlib inline

In [ ]:
plt.style.use('seaborn-notebook')
export_plots = True

### Load the exported metrics from the MXNET Run exported Tensorboard

In [ ]:
def get_hist_metrics(json_dir='./logs/json/'):
    """
    Loads the exported metrics from .json files in a dictionary obj. with 2D numpy arrays
    """
    json_files = glob.glob('%s/*' % (json_dir))
    hist_metrics = {}
    for json_file in json_files:
        
        # extract the metric name using a regular expression
        m = re.search(r'(?<=run_)\w+(?<!.json)', json_file)
        metric_name = m.group(0)
        
        # load the one json file of which each contain a metric
        with open(json_file) as json_data:
            data = json.load(json_data)

        # store the metric data in a 1D-numpy array
        metric_data = np.zeros((len(data), 1))
        for i, row in enumerate(data):
            metric_data[i] = row[2]

        # fill in our dict object using the stored values
        hist_metrics[metric_name] = metric_data
        
    return hist_metrics

In [ ]:
m = re.search(r'(?<=run_)\w+(?<!.json)', './19_res_layers/run_loss_train-tag-loss.json')
metric_name = m.group(0)

In [ ]:
hist_19_res_layers = get_hist_metrics('./19_res_layers/')
hist_15_reslayers_5_layers_stem = get_hist_metrics('./15_reslayers_5_layers_stem/')

```python
total_k_steps = int(hist_lr_0_001_mxnet['train_policy_loss'][-1, 0])+1


metrics_compact = {}

try:
    plt.plot(hist_lr_0_1_keras[metric])
except KeyError:
    plt.plot(hist_lr_0_1_keras[metric.replace('train_', '')])

metric = np.zeros(total_k_steps)

metrics_compact

metrics_compact = {}

for hist in [hist_lr_0_01_mxnet, hist_lr_0_001_mxnet]:
    for metric in hist.keys():
        for type_metric in ['train_', 'val_']:
            if type_metric in metric:
                metric_crop = metric.replace(type_metric, '')
                if metric_crop not in metrics_compact:
                    metrics_compact[metric_crop] = {}
                if type_metric not in metrics_compact[metric_crop]:
                    metrics_compact[metric_crop][type_metric] = np.zeros(total_k_steps)
                #print(metrics_compact)
                for row in hist[metric]:
                    #print(row[0])
                    (metrics_compact[metric_crop][type_metric])[int(row[0])] = row[1]

## Fill in the first values

hist_lr_0_1_keras.keys()

for metric in hist_lr_0_1_keras:
    if 'val_' in metric:
        for i, val in enumerate(hist_lr_0_1_keras[metric]):
            metric_crop = metric.replace('val_', '')
            if metric_crop in metrics_compact:
                if metrics_compact[metric_crop]['val_'][i] == 0:
                    metrics_compact[metric_crop]['val_'][i] = val
    else:  # train metric
        for i, val in enumerate(hist_lr_0_1_keras[metric]):
            if metric in metrics_compact:
                if metrics_compact[metric]['train_'][i] == 0:
                    metrics_compact[metric]['train_'][i] = val
```                    

## Now visualisize the full data

In [ ]:
colors = ['cornflowerblue', 'darkorange']

In [ ]:
for i, metric in enumerate(hist_19_res_layers):
    plt.plot(hist_19_res_layers[metric], '-', label='%s (%s)'%(metric, 'ResnetA0'), c=colors[i])
for i, metric in enumerate(hist_15_reslayers_5_layers_stem):
    plt.plot(hist_15_reslayers_5_layers_stem[metric], '--', label='%s (%s)'%(metric, 'ResnetA0v2'), c=colors[i])           
    #print(metric)
plt.title('Training Comparision:\nResnetA0 (1 Conv - 19 ResLayers)\nResnetA0v2 (4 Conv - 15 ResLayers (LRelu, SE)) \nlr = 0.1, batch_size=1024')
plt.xlabel('Number of Epoch')
apply_def('ResnetA0vsResnetA0v2')

In [ ]:

def apply_def(filename):
    plt.legend()
    plt.grid(axis='y')
    if export_plots is True:
        plt.savefig('./plots/%s.png'%filename, bbox_inches='tight')
        plt.savefig('./plots/%s.pdf'%filename, bbox_inches='tight')
    plt.show()

plt.plot(h)

for metric in metrics_compact:
    for type_set in metrics_compact[metric]:
        plt.plot(metrics_compact[metric][type_set][:-3], label='%s%s'%(type_set, metric)) # [:-3] because the last 3 entries are 0 somehow
        plt.grid(axis='y')
        plt.xlabel('Thousands of Steps / Batch-Size 2048')        
    apply_def(metric)

## Export the structure of the network architecture



import sys
sys.path.insert(0,'../../../../../')

from DeepCrazyhouse.src.training.architecture.mxnet_alpha_zero import alpha_zero_model

import mxnet as mx

# get the symbol defintion of the alpha zero architecture
sym = alpha_zero_model(num_res_blocks=7, n_labels=4992, grad_scale_value=0.01, grad_scale_policy=0.99)
# Visualize your network
pltn = mx.viz.plot_network(sym)
pltn.view('./plots/alpha_zero_model_7x256')